# Магические методы классов

`special methods`, also known as `magic methods` or `dunder methods`

## 1. `__new__` - метод создания (конструктор) класса

```
__new__ is a static method, while __init__ is an instance method.
__new__ is responsible for creating and returning a new instance, while __init__ is responsible for initializing the attributes of the newly created object.
__new__ is called before __init__.
__new__ happens first, then __init__.
__new__ can return any object, while __init__ must return None.
```

In [8]:
class Person:
    def __new__(cls, name, age):
        print("Creating a new Person object")
        instance = super().__new__(cls)
        return instance

    def __init__(self, name, age):
        print("Initializing the Person object")
        self.name = name
        self.age = age

    def __call__(self):
        print('Called')
        return self.age * 2

person = Person("Alina", 19)
print(f"Person's name: {person.name}, age: {person.age}")
person()

Creating a new Person object
Initializing the Person object
Person's name: Alina, age: 19
Called


38

In [16]:
class C:

    def __init__(self, n=0):
        self.n = n

    def __call__(self, a, b):
        print(a, b, self.n)

c = C(1)

In [18]:
c(2,3)

2 3 1


In [2]:
def f():
    print(1)

In [6]:
f.__call__()

1


#### Расширение базовых типов данных

In [27]:
class Distance(float):
    def __new__(cls, value, unit):
        instance = super().__new__(cls, value)
        instance.unit = unit
        return instance

dist = Distance(10.2, 'km')
dist, dist.unit

10.2


(10.2, 'km')

#### Синглтон

In [17]:
class Singleton:
    _instance = None

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

a = Singleton()
b = Singleton()

a == b

True

In [11]:
# создание Синглтонов (Singletone)

class DataBase:
    __instance = None
    
    def __init__(self, user, psw, port):
        self.user = user
        self.psw = psw
        self.port = port

    def __new__(cls, *args, **kwargs):
        if cls.__instance is None:
            cls.__instance = super().__new__(cls)
         return cls.__instance
        
    def connect(self):
        print(f"соединение с БД: {self.user}, {self.psw}, {self.port}")
 
    def close(self):
        print("закрытие соединения с БД")
 
    def read(self):
        return "данные из БД"
 
    def write(self, data):
        print(f"запись в БД {data}")

In [15]:
db = DataBase(user='alina', psw='123', port=9000)
print(1, id(db), db)

1 123258787613088 <__main__.DataBase object at 0x701a6c4049a0>


In [16]:
db = DataBase(user='alina', psw='123', port=9000)
print(1, id(db), db)

1 123258787613088 <__main__.DataBase object at 0x701a6c4049a0>


### Прочие специальные методы

In [28]:
class Box:
    "fjvbndkjfvbn"

    def __init__(self, weight: int):
        self.weight = weight
 
    def __add__(self, other):
        if not isinstance(other, (int, Box)):
            raise ArithmeticError("Правый операнд должен быть типом int или объектом Box")
        weight = other if isinstance(other, int) else other.weight
        self.weight += weight
        return self

    def __eq__(self, other):
        return self.weight == other.weight


Box1 = Box(5)
Box2 = Box(30)
Box3 = Box(5)

print(Box1 == Box2, Box1 == Box3)
box4 = Box1 + Box2 + Box3
print( id(Box1) == id(box4) )
print( Box1 == box4 )
print( box4, box4.weight )
        

False True
True
True
<__main__.Box object at 0x000001D532806900> 40


In [34]:
class A(object):
    pass

In [46]:
MyClass = type('MyClass', (object,), {'foo': 1111})

def f(self, x):
    print(x)


MyClass.f = f

mc = MyClass()
mc.f(99)

99


In [52]:
hash(Box)

125948010004

In [24]:
dir(Box)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

In [25]:
Box1 += Box2

In [26]:
Box1.weight

70

In [23]:
from functools import lru_cache, cache

@cache
def c(x):
    print(x)
    return x+1

In [28]:
c(2)

2


3

In [58]:
class A:
    "aa"

a = A()

if hasattr(a, 'x'):
    print(1)

### `__getattr__`, `__getattribute__`

In [60]:
class B:
    
    def __init__(self, n):
        self.n = n

    def __getattr__(self, name):
        """
        метод __getattr__ определяет поведение,
        когда наш атрибут, который мы пытаемся получить, не найден
        """
        return 'Nothing found :('

    def __getattribute__(self, name):
        """
        метод __getattribute__ вызывается в любом случае, когда мы
        обращаемся к какому-либо атрибуту объекта
        """
        if name == 'attr':
            return 'nope attr'
        else:
            # return object.__getattribute__(self, name)
            return super().__getattribute__(name)

In [66]:
b = B(n='alina')
print(b.name)
b.n

Nothing found :(


'alina'

In [ ]:
a['n']
a.n

In [72]:
class DotDict(dict):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.deleted = {}

    def __getattr__(self, name):
        try:
            return self[name]
        except KeyError:
            self[name] = None

    def __getattribute__(self, name):
        if name == 'attr':
            return 'nope attr'
        else:
            # return object.__getattribute__(self, name)
            return super().__getattribute__(name)

    def __setattr__(self, name, value):
        self[name] = value

    def __delattr__(self, name):
        self.pop(name, None)

    def __getitem__(self, name):
        print('get item')
        return super().__getitem__(name)

    def __setitem__(self, name, value):
        print('set item')
        return super().__setitem__(name, value)

    def __delitem__(self, name):
        print('del item')
        self.deleted[name] = self[name]
        return super().__delitem__(name)

In [76]:
a = DotDict(a=1, b=2)

set item


In [78]:
a['a'] == a.a == 1

get item
get item


True

In [80]:
a.c is None

get item
set item


True

In [5]:
a['b']

get item


In [6]:
a.b = 2

set item


In [118]:
a['b']

get item


2

In [119]:
a.attr

'nope attr'

In [86]:
a['b'] = 3
a['c'] = 3

set item
set item


In [88]:
del a['b']

del item
get item
get item


In [90]:
a.deleted

get item


{'b': 3}

In [125]:
a.pop('c')

3

In [107]:
class Dict:

    def __init__(self, **kwargs):
        self.adict = {**kwargs}

    def __setitem__(self, name, value):
        self.adict[name] = value

    def __getitem__(self, name):
        try:
            return self.adict[name]
        except KeyError:
            raise KeyError(f'Dict не имеет ключа {name}')

    def __delitem__(self, name):
        self.adict.pop(name, None)

    def __repr__(self):
        return str(self.adict)

In [108]:
d = Dict(a=1, b=2)

In [109]:
d

{'a': 1, 'b': 2}

In [110]:
d[3] = 3

In [111]:
d

{'a': 1, 'b': 2, 3: 3}

1. сделать одномерный массив через генератор списков

In [92]:
arr = [[1,2,3], [4,5], [6,7,8,8]]


2. перепаковать одномерный массив в массив с массивами длиной N

In [100]:
N = 3
arr = [i for i in range(24)]

In [ ]:
[[0,1,2], [..]]